In [ ]:
try:
    import google.colab
    inColab = True
except ImportError:
    inColab = False

### Install Library

In [ ]:
# Hugging Face에서 개발한 도구로, 딥러닝 모델의 훈련과 추론 속도를 높이기 위한 최적화
# PEFT( Parameter-Efficient Fine-Tuning) 딥러닝 모델의 파라미터를 효율적으로 미세 조정하는 방법 제공
# bitsandbytes 딥러닝 모델을 더 작은 메모리 풋프린트로 실행할 수 있게 해주는 라이브러리
# transformers 사전 학습된 모델들을 제공하는 도구, 대형 모델을 쉽게 사용할 수 있게 해줌
# trl(Transformers Reinforcement Learning) 사전 학습된 트랜스포머 모델에 강화 학습 적용하여 특정 작업에 맞게 모델을 더 정밀하게 조정하는 데 사용
# dataset 다양한 데이터셋을 쉽게 로드, 처리, 변환, 분석할 수 있는 라이브러리
# -U 옵션: 지정된 패키지가 이미 설치되어 있는 경우에도 최신 버전으로 업그레이드
if inColab == True:
    !pip install -U pandas==2.2.2 numpy==2.0.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# # Hugging Face에서 개발한 도구로, 딥러닝 모델의 훈련과 추론 속도를 높이기 위한 최적화
# # PEFT( Parameter-Efficient Fine-Tuning) 딥러닝 모델의 파라미터를 효율적으로 미세 조정하는 방법 제공
# # bitsandbytes 딥러닝 모델을 더 작은 메모리 풋프린트로 실행할 수 있게 해주는 라이브러리
# # transformers 사전 학습된 모델들을 제공하는 도구, 대형 모델을 쉽게 사용할 수 있게 해줌
# # trl(Transformers Reinforcement Learning) 사전 학습된 트랜스포머 모델에 강화 학습 적용하여 특정 작업에 맞게 모델을 더 정밀하게 조정하는 데 사용
# # dataset 다양한 데이터셋을 쉽게 로드, 처리, 변환, 분석할 수 있는 라이브러리
# # -U 옵션: 지정된 패키지가 이미 설치되어 있는 경우에도 최신 버전으로 업그레이드
# if inColab == True:
#     !pip install -U accelerate==0.32.1 peft==0.12.0 bitsandbytes==0.43.3 transformers==4.44.2 trl==0.9.6 datasets==2.21.0 tensorboard==2.17.1

In [ ]:
# !pip install triton==2.1.0

In [ ]:
# !pip install -U Numpy==1.26.4 pandas==2.2.2 scipy==1.14.1

In [ ]:
import os
import torch ## 딥러닝 라이브러리
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from datetime import datetime
import huggingface_hub

In [ ]:
if inColab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
huggingface_hub.login()

# 1. set model and dataset

### ★★★ 수정 포인트 ★★★

In [ ]:
# set base model path
# base_model = "beomi/Llama-3-Open-Ko-8B"
# base_model = "upstage/SOLAR-10.7B-Instruct-v1.0"
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
# base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# base_model = model_id = "google/gemma-7b-it"
dataset_name = "LEEHEEWON/midTermModel"
datasetCommon = load_dataset(dataset_name, split="train")

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.51k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45 [00:00<?, ? examples/s]

In [ ]:
datasetCommon[1]

{'instruction': 'question으로 질문하면 text안에 있는 형식으로 대답을 하게 해줘!',
 'input': 'ave;new 본거지 어디야',
 'output': '도쿄 치요다구'}

In [ ]:
# 솔라 포맷으로 변환하는 함수
### System:\n{input}\n\n### User:\n{instruction}\n### Assistant:\n

def convert_to_solar_format(dataset):
    instruction = dataset['instruction'] ##문제
    output = dataset['output'] ## 정답

    text = f"<s>### User:\n{instruction}\n\n### Assistant:\n{output}"
    dataset["text"] = text
    return dataset

In [ ]:
# gemma 포맷으로 변환하는 함수 "<start_of_turn>user Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] <end_of_turn>
#<start_of_turn>model # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum <end_of_turn>" https://huggingface.co/google/gemma-7b-it
### System:\n{input}\n\n### User:\n{instruction}\n### Assistant:\n

# def convert_to_gemma_format(dataset):
#     instruction = dataset['instruction']
#     output = dataset['output']

#     text = f"<start_of_turn>{instruction}<end_of_turn><start_of_turn>{output}<end_of_turn>"
#     dataset["text"] = text
#     return dataset

In [ ]:
def convert_to_alpaca_format(dataset):
    instruction = dataset['instruction'] ##문제
    output = dataset['output'] ## 정답
    # 'text' 컬럼 생성
    text = f"### Instruction:\n{instruction}\n\n###Input:\n \n\n###Output:\n{output}"
    dataset["text"] = text
    return dataset

# 'text' 컬럼 생성

In [ ]:
datasetCommon = datasetCommon.map(convert_to_alpaca_format, remove_columns=['instruction', 'input', 'output']) ## 허깅페이스에서 알파카 함수 가져오고 text칼럼 외에 칼럼들은 다 날린다

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [ ]:
datasetCommon

Dataset({
    features: ['text'],
    num_rows: 45
})

In [ ]:
datasetCommon[10] ## 텍스트 값만 남기고 삭제했으므로 텍스트값만 나오게 된다.

{'text': '### Instruction:\nquestion으로 질문하면 text안에 있는 형식으로 대답을 하게 해줘!\n\n###Input:\n \n\n###Output:\nPrism'}

# 2. Config efficient fine-tuning with low-rank adaptation.

In [ ]:
torch.cuda.get_device_capability()[0]

8

In [ ]:
# 현재 사용 중인 GPU의 주요 아키텍처 버전을 반환 8버전 이상 시 bfloat16 활용
# NVIDIA Ampere 아키텍처 이상 시에만 처리
# 정확도를 위하여 float 16이 타도록 강제설정 8->10 ★ 향후 변경 필요
if torch.cuda.get_device_capability()[0] >= 10: ## 10이상이면 falsh_attention은 예전에 쓰던 버전으로 돌아감
    # 고속 attention 메커니즘을 구현하는 라이브러리
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2" ##
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16


# BitsAndBytesConfig 객체활용 양자화 설정
quant_config = BitsAndBytesConfig(
    # 모델을 4비트 양자화하여 로드할지 여부 결정
    load_in_4bit=True,
    # 양자화 방법 (nf4: Non-Uniform Quantization, "nf4","fp16 등))
    bnb_4bit_quant_type="nf4",
    # (4비트 양자화 시 사용할 데이터 타입, "torch.float16, bfloat16, float32 등)
    bnb_4bit_compute_dtype=torch_dtype,
    # 이중 양자화 사용여부 (이중 양자화는 양자화 과정에서 정밀도 높이기 위해 활용, 대신 더 연산은 복잡)
    bnb_4bit_use_double_quant=False,
)

# 3. Load Pre-trained Language Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    # 불러올 모델 정의
    base_model,
    # 모델 양자화 설정값
    quantization_config=quant_config,
    # 모델의 레이어를 할당할 장치 ("":0 -> 전체 모델을 GPU 0에 할당, "auto"는 알아서, "{"layer_0":0, ... 형태로 레이어별 할당 가능)
    #device_map={"": 0}
    device_map="auto"
)
# 캐시 사용 여부 (모델 출력 매번 새로 계산)
# 새로운 데이터에 대한 계산 시 저장하는게 불필요함 또한 True 시 캐시 저장할 메모리 사용하여 메모리 사용량 증가함
model.config.use_cache = False
# 모델읜 pretraining tensor parallelism 설정 1인경우 병렬처리 안하고 단일장치 활용 2이상 시 병렬 처리를 분산 가능
# 병렬 처리 시 추가적인 메모리 오버헤드 발생 가능함
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# 4. Load Pre-trained Language Model Tokenizer

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# Must add EOS_TOKEN at response last line
tokenizer.pad_token = tokenizer.eos_token

# ★수정 포인트!!! 기존 # tokenizer.padding_side = "right"
EOS_TOKEN = tokenizer.eos_token
def prompt_eos(sample):
    sample['text'] = sample['text']+EOS_TOKEN
    return sample
datasetCommon = datasetCommon.map(prompt_eos) ## eot토큰이 있어야 답변을 끊어서 할 수 있다.

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [ ]:
datasetCommon[1]

{'text': '### Instruction:\nquestion으로 질문하면 text안에 있는 형식으로 대답을 하게 해줘!\n\n###Input:\n \n\n###Output:\n도쿄 치요다구<|eot_id|>'}

In [ ]:
import gc
# # Flush memory
# del trainer, model
gc.collect() ## 쓰레기값들을 모아서 버린다
torch.cuda.empty_cache()

# 5. Config training parameter for LoRA (Parameter-Efficient Fine-Tuning (PEFT)



In [ ]:
peft_params = LoraConfig(
    # lora 방식의 튜닝에서 저차원 병렬 레이어 학습 ( 값이 크면 저차원 병렬 레이어 영향이 커지고 작으면 줄어듬, 기본 16, 1~128)
    lora_alpha=16,
    # 과적합 방지 (0.1에서 증가 시킴, 0.0~0.3)
    lora_dropout=0.1,
    # LoRA 저차원 공간의 차원 수 (값이 커지면 파라미터 공간 커지지만 계산비용 증가 4, 8, 16, 32, 64 로 튜닝, 4~128)
    r=64,
    # LoRA 모델의 바이어스 파라미터 적용여부 None = 미적용 ("none" or "all", "lora_only" 설정 가능)
    bias="none",
    # LoLA 모델의 작업 유형 (GPT와 같은 인과모델) ("CASUAL_LM": GPT, "SEQ_CLASSSIFICATION": 시퀀스 분류, "TOKEN_CLASSIFICATION": 토큰 분류, "SEQ_2_SEQ_LM": 번역모델)
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    # 학습 결과와 체크포인트를 저장할 디렉토리 경로
    output_dir="./results",
    # 전체 데이터셋을 반복학습 주기 (1~50 정도 활용)
    num_train_epochs = 5,
    # 각 GPU 장치에서 사용하는 배치 크기 (배치 크기에 따라 가중치 업데이트 1~64)
    # 배치크기는 모델히 한번에 처리하는 데이터 샘플의 수
    per_device_train_batch_size=3,
    #그래디언트 누적을 통한 배치 크기 증가 (여러 작은 배치를 합쳐서 효과적으로 큰 배치 크기를 적용 가능) *4 적용 시 그래디언트 누적 4개 후 한번에 가중치 업데이트
    # 배치크기가 4배 증가한 효과를 가지지만 실제 메모리 사용은 배치 1개 크기만 사용
    gradient_accumulation_steps=1,
    # 사용할 옵티마이저 설정 ("adamw, paged_adamw_32bit","adamw_torch")
    optim="paged_adamw_32bit",
    # 학습 중 모델 체크포인트 저장 주기 (스템 수)
    save_steps=25,
    # 학습 중 로그를 기록할 주기(스텝 수)
    logging_steps=25,
    # 학습률 설정 (모델의 파라미터 업데이트 시 스템 크기, 1e-6 ~ 1e-2))
    learning_rate=2e-4,
    # 모델의 가중치에 패널티 적용하여 과적합 방지 (0.0은 정규화 안함, 0.0 ~ 0.1)
    weight_decay=0.001,
    # 16비트 부동 소수점(FP16) 연산을 사용하여 메모리 사용량과 계산속도 개선 (사용시 정밀도 감소, 32비트 부동소수점보다 절반)
    fp16=False,
    # bfloat16 연산을 사용하여 학습 FP16대비 더 넓은 지수 범위 (사용시 정밀도 감소, 32비트 부동소수점보다 절반)
    bf16=False,
    # (그래디언트 클리핑을 위한 최대 노름 값 설정, 0.1 ~ 10)
    # 그래디언트 클리핑은 그래디언트의 크기가 너무 커서 학습이 불안정해지는 것을 방지하기 위한 기법 0.3 초과하지 않도록 함 (크면 가중치가 너무 크게업데이트됨)
    max_grad_norm=0.3,
    # 전체 학습단계 수 (-1 인 경우 epochs, 이외에는 스탭수,  -1~100000)
    max_steps=-1,
    # 학습률 warmup비율 설정 (학습 초기에 학습률을 서서히 증가시켜 안정적인 학습, 0.0~0.5)
    # 너무 높으면 학습률이 너무 늦어짐
    warmup_ratio=0.03,
    # 배치 내 시쿼스 길이를 그룹화하여 패딩을 최소화 (데이터 시권스길이 유사한것 끼리 그룹화하여 메모리 효율설 높임)
    group_by_length=True,
    # 학습률 스케줄러의 유형 설정 ("linear","cosine","constant","polynomial" 등) * constant는 학습률을 일정하게 유지
    lr_scheduler_type="constant",
    # 학습 로그를 보고할 플랫폼 설정 (아웃풋 디렉토리 참고) -> tensorboard --logdir=./results/runs * wandb로 설정 가능
    report_to="tensorboard"
)


# 6. Train Model

In [ ]:
# from datasets import Dataset
# text_list = datasetCommon["text"]  # 또는 datasetCommon.map(lambda x: x["text"])

# # Dataset 객체로 변환 (필요시)
# train_dataset = Dataset.from_dict({"text": text_list})
# train_dataset

In [ ]:
trainer = SFTTrainer(
    # 학습할 모델
    model=model,
    # 모델 학습에 사용할 데이터셋
    train_dataset=datasetCommon,
    #  PEFT(파라미터 효율적 미세 조정) 설정 정의
    peft_config=peft_params,
    # 데이터셋에서 학습 데이터셋 텍스트 필드 이름
    # dataset_text_field="text",
    # 입력 시퀀스 최대 길이 (128 ~ 1024) * 길이가 길수록 더많은 컨텍스트를 모델에 제공가능 단, 메모리 사용량 증가
    # max_seq_length=None,
    # 모델과 함께 사용할 토크나이저
    # tokenizer=tokenizer,
    args=training_params,
    # 입력 시퀀스 패킹여부 (패킹 시 짧고/긴 시퀀스를 혼합하여 배치 처리 서능 개선)
    # packing=False,
)
trainer.train()

Converting train dataset to ChatML:   0%|          | 0/45 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/45 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/45 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/45 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
25,2.488400
50,0.960000
75,0.666200


TrainOutput(global_step=75, training_loss=1.3715082550048827, metrics={'train_runtime': 41.1336, 'train_samples_per_second': 5.47, 'train_steps_per_second': 1.823, 'total_flos': 468155591073792.0, 'train_loss': 1.3715082550048827})

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=./results/runs

# 7. Verify

In [ ]:
# text-generation 작업을 위한 파이프라인 객체 생성
pipe = pipeline(
    # 작업 유형: 텍스트 생성
    task="text-generation",
    # 사용할 모델
    model=model,
    # 모델의 토크나이저
    tokenizer=tokenizer,
    # 모델에 전달할 추가 인자: float16 데이터 타입을 사용하여 모델을 로드 ("bfloat16"으로 학습했다면 bfloat16 사용)
    model_kwargs={"torch_dtype": torch.float16},
    # 입력 텍스트가 너무 길 경우 잘라내기
    truncation=True
)

Device set to use cuda:0


In [ ]:
def extract_response_llama3(question):
    # 사용자 질문을 포함하는 메시지 리스트를 생성
    messages = [
        {"role": "system", "content": ""},       # 시스템 메시지: 일반적으로 시스템의 지침이나 상태값 저장
        {"role": "user", "content": question},   # 사용자 메시지: 사용자 질문을 포함
    ]
    # 메시지를 토크나이저를 사용하여 모델의 입력 형식에 맞게 변환
    prompt = pipe.tokenizer.apply_chat_template(
        messages,                             # 메시지 리스트
        tokenize=False,                       # 토크나이즈를 하지 않음
        add_generation_prompt=True            # 텍스트 생성을 위한 프롬프트 추가
    )
    # 텍스트 생성 종료 토큰 ID 목록
    terminators = [
        pipe.tokenizer.eos_token_id,                        # End Of Sequence 토큰 ID
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # 사용자 정의 종료 토큰 ID
    ]
    outputs = pipe(
        prompt,                          # 생성할 프롬프트
        max_new_tokens=256,              # 생성할 최대 토큰 수
        eos_token_id=terminators,        # 텍스트 생성 종료를 위한 토큰 ID
        do_sample=False,                  # 샘플링을 사용하여 다음 토큰(텍스트) 생성 (True 시 토큰 무작위 선택 하여 창의성  높임, False 시 창의성 낮춤)
        temperature=0.1,                 # 샘플링의 온도 설정: 낮은 온도는 자유도를 주지 않음 (0보다 크고1보다 작아야함)
        # top_p=0.9,                       # 상위 90%의 확률을 가진 토큰중에서만 무작위로 선택 do_sample true인 경우에만 설정
        num_return_sequences=1           # 생성할 시퀀스의 수: 한 개만 생성
    )
    generated_text = outputs[0]['generated_text']        # 생성된 텍스트를 추출
    response_lines = generated_text.strip().split('\n')  # 텍스트를 줄 단위로 분리
    meaningful_response = response_lines[-1]             # 마지막 줄을 응답으로 선택
    return meaningful_response

In [ ]:
question = "프톨레마이오스 왕조는 아우구스투스 때부터 누구의 지배를 받았어"
response = extract_response_llama3(question)
print(response)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


프톨레마이오스 왕조는 로마 제국에 종속된 국가로, 프톨레마이오스 12세가 죽은 후인 30년경부터 로마 제국에 의해 파괴되기까지 380년간의 기간 동안 로마 제국의 지배를 받았다.


In [ ]:
question = "미국 대통령 선거는 어떤 방식으로 진행 돼"
response = extract_response_llama3(question)
print(response)

이러한 방식으로 미국 대통령 선거가 진행되는 데, 이 방식


In [ ]:
# def generate_text_gemma(model, tokenizer, prompt, device="cuda:0"):
#     # 모델을 디바이스에 로드
#     # model.to(device)

#     # 프롬프트 인코딩
#     inputs = tokenizer(prompt, return_tensors="pt").to(device)

#     # 텍스트 생성
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=512,             # 생성할 최대 토큰 수
#         do_sample=True,                 # 샘플링 사용 여부
#         temperature=0.1,                # 샘플링 온도
#         eos_token_id=tokenizer.eos_token_id  # 종료 토큰 ID
#     )

#     # 생성된 텍스트 디코딩
#     generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return generated_text

# # 예시: 프롬프트를 입력하여 텍스트 생성
# prompt = "<start_of_turn>유튜브 채널 hkcode는 누가 운영하나요?\n<end_of_turn>\n<start_of_turn>\n"
# result = generate_text_gemma(model, tokenizer, prompt)
# print(result)

### ★★★ 수정 포인트 ★★★

In [ ]:
now_str = datetime.now().strftime("%Y_%m_%d_%H")
now_str

'2025_05_02_09'

In [ ]:
if inColab == True:
    savePath = f"/content/gdrive/MyDrive/김효관교수님/김효관교수님/Colab Notebooks/fintech_edu_2025/4.LLM_파인튜닝{now_str}"
    trainer.save_model(savePath)
else:
    savePath = "./models/llama_tune_common"
    trainer.save_model(savePath)

In [ ]:
!pwd

/content/gdrive/MyDrive/김효관교수님/김효관교수님/Colab Notebooks/fintech_edu_2025/4.LLM_파인튜닝


In [ ]:
%cd /content/gdrive/MyDrive/김효관교수님/김효관교수님/Colab Notebooks/fintech_edu_2025/4.LLM_파인튜닝

/content/gdrive/MyDrive/김효관교수님/김효관교수님/Colab Notebooks/fintech_edu_2025/4.LLM_파인튜닝
